In [133]:
import time
import urllib.request
from urllib.request import urlopen
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import random
from urllib.error import HTTPError
from selenium.common.exceptions import NoSuchElementException
import re
import numpy as np
from selenium.webdriver.chrome.options import Options
import pymysql
import os
from openpyxl import Workbook

In [134]:
#수집하고 싶은 태그명
tag = '우울'

#이미지 저장 폴더 경로 입력
directory = "./우울/"

In [135]:
options = Options()
# chrome을 전체화면으로 넓히는 옵션입니다.
options.add_argument('--start-fullscreen')

url = "https://www.instagram.com/explore/tags/" + tag + "/" #행복스타그램,감동

# executable_path에는 chromedriver 실행 파일의 경로를 넣고, chrome_options에는 options 변수를 넣습니다.
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.implicitly_wait(3)
driver.get(url)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [136]:
result = []
pagedown = 0
post_num = 1
collect_driver = webdriver.Chrome()
while True: #끝까지 내릴부분을 찾아라
    bottom = driver.find_element_by_class_name('ixdEe')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    source = soup.find_all('div',{'class':'v1Nh3 kIKUG _bz0w'})
    #source = np.array(source)
    
    if pagedown == 0:
        source = source[9:33]
    elif pagedown == 1:
        source = source[33:45]
    elif pagedown == 2:
        source = source[42:54]
    else:
        source = source[42:]
         
    for i in range(len(source)):
        if(source[i].find('img').get('alt') is not None):
            if('사람' in source[i].find('img').get('alt')):
                try:
                    collect_driver.implicitly_wait(3)
                    #페이지 열기
                    collect_driver.get('https://www.instagram.com' + (source[i].find("a")).get('href'))
                    #페이지 소스 가져오기
                    collect_soup = BeautifulSoup(collect_driver.page_source,'html.parser')
                    #게시글 작성자 이름
                    user_id = collect_driver.find_element_by_class_name('_6lAjh').text
                    #좋아요 개수
                    like = collect_driver.find_element_by_class_name('Nm9Fw').text
                    like_count = like.split(' ')[1]
                    #게시글 생성날짜
                    post_date = collect_soup.find('time',{'class':'_1o9PC Nzb55'}).get('datetime')             
                    #해시태그
                    tag_source = collect_soup.find_all('meta',{'property':'instapp:hashtags'})
                    tag_list = []
                    for j in range(len(tag_source)):
                        tag_list.append(tag_source[j].get('content'))
                    tag_string = "|".join(tag_list)
                    #댓글
                    comment_list = []
                    comment_source = collect_soup.find_all('div',{'class':'C4VMK'})
                    for k in range(1,len(comment_source)):
                        comment = comment_source[k].find('span').text
                        comment = comment.replace("'",";")
                        c_id = comment_source[k].find('a').text
                        comment_list.append([comment,c_id])
                    #텍스트 본문
                    text_source = collect_soup.find('div',{'class':'C4VMK'})
                    text = text_source.find('span').text
                    text = text.replace("'",";")

                    #여러개 이미지 가져오기
                    img = collect_soup.find_all('div',{'class':'KL4Bh'})
                    if img == []:
                        continue
                    img_list = []
                    
                    alt = img[0].find('img').get('alt')
                    alt = alt.replace("'",";")
                    src = img[0].find('img').get('src')
                    img_name = str(post_num)
                    try:
                            #폴더가 없으면 생성해서 저장 
                        if not os.path.exists(directory):
                            os.makedirs(directory)
                        urllib.request.urlretrieve(src, directory + img_name +'.jpg')

                    except HTTPError:
                        print(i+1,"번째 이미지 삭제됨")
                        continue
                    img_list.append([str(post_num),alt])

                    if post_num % 100 == 0:
                        print(post_num)

                    result.append([tag,img_list[0][0],text,tag_string])   

                    post_num +=1

                except NoSuchElementException:
                    print(post_num,"번째 페이지 삭제됨")
                    continue
                
            

    driver.execute_script('arguments[0].scrollIntoView(true);',bottom)
    time.sleep(3)
    pagedown += 1

#드라이버 종료
driver.close()
collect_driver.close()

56 번째 페이지 삭제됨
90 번째 페이지 삭제됨
100
112 번째 페이지 삭제됨
200
300


KeyboardInterrupt: 

In [137]:
write_wb = Workbook()
write_ws = write_wb.active
for i in result:
    write_ws.append(i)
    
write_wb.save('./우울.xlsx')